In [1]:
from ultralytics import YOLO
import sys

# Ensure the project root is on the Python path
sys.path.append('E:/ECE208PROJ/HE2L-Net')
from configs import cfgs
import torch
import os
from matplotlib import pyplot as plt
import cv2

In [2]:
model = YOLO('E:\ECE208PROJ\HE2L-Net\weights\yolo_weights.pt')
image_root = cfgs.TRAIN_Rec.DATASET.test_img
print(image_root)
image_name = 'presentimage3'
image_path = os.path.join(image_root, image_name + '.jpg')
pred_result = model.predict(image_path, show=True, save=True)[0]

E:\ECE208PROJ\HE2L-Net\sampleimages

image 1/1 E:\ECE208PROJ\HE2L-Net\sampleimages\presentimage3.jpg: 192x640 1 \lim_, 1 \to, 1 \frac, 1 +, 3 -s, 2 6s, 1 9, 1 1, 3 vs, 234.6ms
Speed: 1.1ms preprocess, 234.6ms inference, 847.8ms postprocess per image at shape (1, 3, 192, 640)
Results saved to runs\detect\predict19


In [3]:
normalized_boxes = []
for box in pred_result.boxes.data:
    # Extract tensor and convert to numpy array
    x1, y1, x2, y2, conf, cls = box.cpu().numpy()
    
    # Normalize coordinates
    x1_normalized = x1 / pred_result.orig_shape[0]
    y1_normalized = y1 / pred_result.orig_shape[1]
    x2_normalized = x2 / pred_result.orig_shape[0]
    y2_normalized = y2 / pred_result.orig_shape[1]
    
    # Store the normalized box
    normalized_box = (x1_normalized, y1_normalized, x2_normalized, y2_normalized, conf,cls)
    normalized_boxes.append(normalized_box)

In [4]:
# Example input: bbox_coordinates and classes should be populated with your data
bbox_coordinates = [box[:4] for box in normalized_boxes]  # Extract x1, y1, x2, y2
classes = [box[5] for box in normalized_boxes]  # Extract the class index

# Convert lists to tensors
bbox_tensor = torch.tensor(bbox_coordinates, dtype=torch.float32)  # Bounding box tensor
class_tensor = torch.tensor(classes, dtype=torch.long)  # Class ID tensor

# Add an additional dimension using unsqueeze (to simulate batch)
bbox_tensor = bbox_tensor.unsqueeze(0)  # Shape now [1, N, 4]
class_tensor = class_tensor.unsqueeze(0)  # Shape now [1, N]


C:\Users\oscar\AppData\Local\Temp\ipykernel_5092\300261247.py:7: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  class_tensor = torch.tensor(classes, dtype=torch.long)  # Class ID tensor


In [5]:
import sys
# sys.path.append('E:\\ECE208PROJ\\HE2L-Net\\model')
from model.transformer import TransformerModel, pad_or_truncate_output
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
total_tokens = 60
num_special_tokens = 0
model = TransformerModel(num_tokens=total_tokens + num_special_tokens, num_special_tokens=num_special_tokens, d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6)
model = model.to(device)
if not cfgs.TRAIN_Com.start_new:
    checkpoint_path = "E:\\ECE208PROJ\\HE2L-Net\\weights\\transformer_weights.pth"
    checkpoint = torch.load(checkpoint_path, map_location=device)  # Load checkpoint
    model.load_state_dict(checkpoint)

d:\ANACONDA\envs\transformer\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [7]:

output = model(class_tensor,class_tensor, bbox_tensor,gt=None, visualize=False, src_mask=None) 
# Decoding output to LaTeX
latex_output = model.tensor_to_latex(output)
print("Predicted LaTeX:", latex_output)

Predicted LaTeX: ['\\lim_^}^}^^^}}}}{^']
